In [2]:
#final
import pandas as pd
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Load the dataset
file_path = '/mnt/d/Algo Code/datasetNew.csv'
df = pd.read_csv(file_path)

# Extract resource constraints (last row)
constraint_row = df.iloc[-1, 3:].fillna(0).astype(int).values
df = df.iloc[:-1]

# Extract items and groups
items = []
groups = df['Group'].unique()
num_resources = len(constraint_row)

# Create a list of items with their resource usage and values
for _, row in df.iterrows():
    item = {
        'Group': row['Group'],
        'Item': row['Item'],
        'Value': row['Value'],
        'Resources': row[3:].values.astype(int)
    }
    items.append(item)

# Function to generate an initial solution
def initial_solution(items):
    solution = []
    groups = set(item['Group'] for item in items)
    for group in groups:
        group_items = [item for item in items if item['Group'] == group]
        solution.append(np.random.choice(group_items))
    return solution

# Function to evaluate the solution
def evaluate_solution(solution, constraints):
    total_value = sum(item['Value'] for item in solution)
    total_resources = np.sum([item['Resources'] for item in solution], axis=0)
    valid = np.all(total_resources <= constraints)
    penalty = np.sum(np.maximum(total_resources - constraints, 0))  # Penalty for exceeding constraints
    return total_value - penalty if valid else -1

# Function to generate a neighboring solution
def neighbor_solution(solution, items):
    new_solution = solution.copy()
    groups = list(set(item['Group'] for item in solution))
    group = np.random.choice(groups)
    group_items = [item for item in items if item['Group'] == group]
    
    if len(group_items) > 1:
        new_item = np.random.choice(group_items)
        idx = next(i for i, item in enumerate(new_solution) if item['Group'] == group)
        new_solution[idx] = new_item

        # Additionally, swap items between different groups
        other_groups = [g for g in groups if g != group]
        if other_groups:
            other_group = np.random.choice(other_groups)
            other_group_items = [item for item in items if item['Group'] == other_group]
            if other_group_items:
                swap_idx = np.random.choice([i for i, item in enumerate(new_solution) if item['Group'] == other_group])
                new_solution[swap_idx] = np.random.choice(other_group_items)
    
    return new_solution

# Simulated Annealing algorithm implementation
def simulated_annealing(items, constraints, initial_temp=1000, cooling_rate=0.003, max_iterations=1000):
    current_solution = initial_solution(items)
    best_solution = current_solution
    current_value = evaluate_solution(current_solution, constraints)
    best_value = current_value
    temp = initial_temp
    
    for _ in range(max_iterations):
        new_solution = neighbor_solution(current_solution, items)
        new_value = evaluate_solution(new_solution, constraints)
        
        if new_value > current_value or np.random.rand() < np.exp((new_value - current_value) / temp):
            current_solution = new_solution
            current_value = new_value
            if new_value > best_value:
                best_solution = new_solution
                best_value = new_value
        
        temp *= (1 - cooling_rate)
    
    return best_solution, best_value

# Run Simulated Annealing
best_solution, best_value = simulated_annealing(items, constraint_row)

# Output results
print("Best Solution:")
for item in best_solution:
    print(f"Group: {item['Group']}, Item: {item['Item']}, Value: {item['Value']}")

print("\nBest Total Value:", best_value)
print("Resource Constraints:", constraint_row)


/tmp/ipykernel_1883/3805708551.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  constraint_row = df.iloc[-1, 3:].fillna(0).astype(int).values


Best Solution:
Group: Group_53, Item: Item_15, Value: 67.0
Group: Group_75, Item: Item_17, Value: 87.0
Group: Group_20, Item: Item_19, Value: 70.0
Group: Group_27, Item: Item_3, Value: 165.0
Group: Group_49, Item: Item_24, Value: 197.0
Group: Group_95, Item: Item_12, Value: 199.0
Group: Group_65, Item: Item_4, Value: 136.0
Group: Group_89, Item: Item_30, Value: 183.0
Group: Group_77, Item: Item_22, Value: 188.0
Group: Group_25, Item: Item_3, Value: 173.0
Group: Group_68, Item: Item_24, Value: 119.0
Group: Group_52, Item: Item_6, Value: 199.0
Group: Group_45, Item: Item_13, Value: 75.0
Group: Group_63, Item: Item_18, Value: 49.0
Group: Group_69, Item: Item_29, Value: 175.0
Group: Group_35, Item: Item_4, Value: 106.0
Group: Group_26, Item: Item_12, Value: 113.0
Group: Group_93, Item: Item_17, Value: 55.0
Group: Group_37, Item: Item_13, Value: 174.0
Group: Group_58, Item: Item_18, Value: 189.0
Group: Group_6, Item: Item_5, Value: 107.0
Group: Group_97, Item: Item_11, Value: 174.0
Group: G